# 添加AtenToMindSpore算子映射关系教程

`Linux` `Ascend` `GPU` `CPU` `模型迁移` `Pytorch` `ATen`

[![](https://gitee.com/mindspore/docs/raw/r1.9/resource/_static/logo_source.png)](https://gitee.com/mindspore/mindinsight/blob/r1.9/ecosystem_tools/mindconverter/tutorial/add_aten2mindspore_operator_mapper_tutorial.ipynb)

## 概述

MindConverter API 基于PyTorch源生计算图[TorchScript](https://pytorch.org/docs/stable/jit.html)进行模型迁移。通过将TorchScript中的[ATen算子](https://pytorch.org/cppdocs/index.html#aten)映射到MindSpore中的对应数学实现的算子，来生成MindSpore模型脚本和权重文件。因此ATen算子到MindSpore算子的映射关系的正确性是决定迁移结果正确与否的关键要素之一。

## 环境准备

本案例需要安装以下依赖库：

```bash
pip install mindspore==1.6.0
pip install torch==1.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
```

## 自定义算子映射基类

```python
class AtenToMindSporeMapper(...):
    """实现ATen算子到MindSpore算子的基类。"""

    @staticmethod
    def _operation_name_in_ms(*args, **kwargs) -> str:
        """返回MindSpore算子名。"""

    @staticmethod
    def _convert_trainable_weights(**kwargs) -> dict:
        """返回由ATen算子中的权重信息转换后的MindSpore算子中的权重。"""

    @staticmethod
    def _generate_snippet_template(**kwargs) -> (dict, dict, list, tuple):
        """
        返回用于生成MindSpore模型脚本的信息。

        Returns:
            Dict, template.  # 分别保存MindSpore模型脚本中的`__init__`和`construct`部分的算子代码。
                如果template =
                {
                    variable_slot: {
                        init: ["init_code_0", "init_code_1",...],
                        construct: ["construct_code_0", "construct_code_1",...]
                    }
                }，
                那么，该算子在MindSpore模型脚本中的代码为：
                class A(nn.Cell):
                    def __init__(self):
                        super(A, self).__init__()
                        init_code_0
                        init_code_1
                        ...

                    def construct(self, x):
                        construct_code_0
                        construct_code_1
                        ...
            Dict, exchange_msg.  # 保存来自于其他算子的与该算子有关的信息，例如：该算子的输入来自于其他算子的输出。
                exchange_msg =
                {
                    variable_slot: {
                        operation: op_name,  # MindSpore算子名。
                        variable_name: None,  # 算子在MindSpore模型脚本中的引用变量名，由后续模块生成，此处设置为None。
                        output_type: "tensor",  # 该算子的输出类型，通常设置为张量即可。
                        inputs: [],  # 该算子的真实输入，由后续模块生成，此处设置为list()。
                        args: args,  # 该算子的参数。
                        weights: weights,  # 该算子包含的全部张量，包括静态张量与权重张量。
                        trainable_params: trainable_params,  # 该算子的权重信息，后续会保存在CheckPoint file中。
                        parameters_declared: parameters_declared,  # 与trainable_params配合使用，在MindSpore模型脚本中加载非算子权重的静态张量。该字段可能不存在。
                        group_inputs: group_inputs  # 与inputs配合使用，划分该算子来自于其他算子输出的输入变量。该字段可能不存在。
                    }
                }
            List, outputs_list.  # 该算子的输出名。算子可能存在多个输出，因此用列表封装。
            Tuple, outputs_mapping.  # ATen算子与MindSpore算子的输出的对应关系，主要用于都输出算子的映射。
        """

    @staticmethod
    def get_args_name_list(**kwargs) -> list:
        """
        返回有效的ATen算子参数列表。
        同一个ATen算子可能存在多种声明，通过参数个数和参数类型进行区分，从而适配前端算子的灵活表达。
        以`aten::zeros`为例：
            at::Tensor at::zeros(at::IntArrayRef size, at::TensorOptions options = {})
            at::Tensor at::zeros(at::IntArrayRef size, c10::optional<at::DimnameList> names, at::TensorOptions options = {})
            at::Tensor at::zeros(at::IntArrayRef size, c10::optional<at::ScalarType> dtype, c10::optional<at::Layout> layout, c10::optional<at::Device> device, c10::optional<bool> pin_memory)
            at::Tensor at::zeros(at::IntArrayRef size, c10::optional<at::DimnameList> names, c10::optional<at::ScalarType> dtype, c10::optional<at::Layout> layout, c10::optional<at::Device> device, c10::optional<bool> pin_memory)
        那么，kwargs["args_name"] =
            {
                2: ['shape', 'unused'],
                3: ['shape', 'unused', 'unused'],
                5: ['shape', 'dtype', 'unused', 'unused', 'unused'],
                6: ['shape', 'unused', 'dtype', 'unused', 'unused', 'unused']
            }

        Returns:
            List, args_name_list.  # 通过参数数量确定的ATen算子参数名列表。
        """

    @staticmethod
    def _params_parser(raw_params, args_name, trainable_params) -> (list, dict, list):
        """
        对ATen算子的参数进行解析，获取用于生成MindSpore对应算子的信息。

        Args:
            raw_params (dict): ATen算子的参数。
            args_name (Union<list, dict>): ATen算子的预设参数名列表。仅有一个声明时为列表(list)，有多个声明时为字典(dict)。
            trainable_params (dict): ATen算子的张量信息。

        Returns:
            list, inputs.  # ATen算子的参数在MindSpore模型脚本中的对应表达形式。
            dict, args.  # ATen算子中参数的值。
            list, group_inputs.  # 与`inputs`对应，用于正确地生成MindSpore算子中来自于其他算子输出的输入变量。
        """
```

## 自定义添加算子映射关系

确定ATen算子和MindSpore算子的对应关系，通常需要对于算子的数学实现有一定的了解。

|算子类型|信息来源|说明|
|:-----:|:-----:|:-----:|
|`aten`|[API文档](https://pytorch.org/cppdocs/index.html#aten)<br>[ATen算子注册表](https://github.com/pytorch/pytorch/blob/master/aten/src/ATen/native/native_functions.yaml)|优先使用API文档中的`Functions`和`Class Tensor`中的声明。|
|`mindspore`|[API文档](https://www.mindspore.cn/docs/zh-CN/r1.9/index.html)|MindSpore网络中所需算子主要在`mindspore.nn`, `mindspore.ops`和`mindspore.numpy`中。|

通过查询ATen算子的API文档，获取`aten::zeros`的声明：

```text
at::Tensor at::zeros(at::IntArrayRef size, at::TensorOptions options = {})
at::Tensor at::zeros(at::IntArrayRef size, c10::optional<at::DimnameList> names, at::TensorOptions options = {})
at::Tensor at::zeros(at::IntArrayRef size, c10::optional<at::ScalarType> dtype, c10::optional<at::Layout> layout, c10::optional<at::Device> device, c10::optional<bool> pin_memory)
at::Tensor at::zeros(at::IntArrayRef size, c10::optional<at::DimnameList> names, c10::optional<at::ScalarType> dtype, c10::optional<at::Layout> layout, c10::optional<at::Device> device, c10::optional<bool> pin_memory)
```

通过查询MindSpore算子的API文档，获取具有相似数学实现的MindSpore算子`mindspore.numpy.zeros`的声明：

```text
mindspore.numpy.zeros(shape, dtype=mstype.float32)
    Returns a new tensor of given shape and type, filled with zeros.

    Parameters
        shape(Union[int, tuple, list]) - the shape of the new tensor.
        dtype(Union[mindspore.dtype, str], optional) - Designated tensor dtype. Default is mstype.float32.

    Returns
        Tensor, with the designated shape and dtype, filled with zeros.

    Raises
        TypeError - If input arguments have types not specified above.
        ValueError - If shape entries have values < 0.
```

### 根据`aten::zeros`和`mindspore.numpy.zeros`的声明实现映射。

In [1]:
# 固定导入，多数情况下不需要修改。
from mindconverter.graph_based_converter.common.utils import reset_template_and_exchange_msg
from mindconverter.graph_based_converter.constant import WeightType, PYTORCH_MS_MAP
from mindconverter.graph_based_converter.mapper.base import AtenToMindSporeMapper


class ZerosMapper(AtenToMindSporeMapper):
    """定义映射关系类，该类名与ATen算子保持一致。"""

    @staticmethod
    def _operation_name_in_ms(*args, **kwargs):
        """返回与ATen算子对应的MindSpore算子名。其中的`mindspore.numpy`已重命名为`ms_np`，不在此处返回。"""
        return "zeros"

    @staticmethod
    def _convert_trained_weights(**kwargs):
        """返回该算子的张量信息。"""
        weights = kwargs.get("weights", list())
        args_name = {
            2: ["shape", "unused"],
            3: ["shape", "unused", "unused"],
            5: ["shape", "dtype", "unused", "unused", "unused"],
            6: ["shape", "unused", "dtype", "unused", "unused", "unused"]
        }
        # 获取该算子的参数名列表。
        args_name_list = ZerosMapper.get_args_name_list(**kwargs, args_name=args_name)
        # 获取该算子张量信息。
        trainable_params = dict()
        for weight in weights:
            trainable_params[args_name_list[weight.location]] = {
                "data": weight.value, "location": weight.location,
                "type": WeightType.PARAMETER.value, "onnx_name": weight.name
            }
        return trainable_params

    @staticmethod
    def _generate_snippet_template(**kwargs):
        """返回用于生成MindSpore模型脚本中的该算子代码所需要的信息。"""
        # 固定代码段，不需要修改。
        template, exchange_msg, outputs_list, outputs_mapping = AtenToMindSporeMapper._generate_snippet_template(
            **kwargs)
        raw_params = kwargs.get("raw_params")
        if not raw_params:
            return template, exchange_msg, outputs_mapping, outputs_mapping

        op = kwargs.get("operation")

        variable_slot = "var_0"
        trainable_params = kwargs.get("trainable_params", dict())

        # ATen算子的参数名配置。
        args_name = {
            2: ["shape", "unused"],
            3: ["shape", "unused", "unused"],
            5: ["shape", "dtype", "unused", "unused", "unused"],
            6: ["shape", "unused", "dtype", "unused", "unused", "unused"]
        }

        # 获取算子对应的参数列表。
        # ATen算子中的`shape`参数是一个元组，该元组可能是静态配置，也可能是来自于其他算子输出的动态输入。
        # 当其为动态输入时，则`shape = (shape0, shape1, shape2,...)`。
        # if return_raw:
        #     return [(shape0, shape1, shape2,...), "unused"]
        # else:
        #     return [shape0, shape1, shape2,..., "unused"]
        raw_args_name_list = ZerosMapper.get_args_name_list(params=raw_params, args_name=args_name, return_raw=True)

        # 解析ATen算子参数，获取MindSpore算子对应的表达。
        inputs, args, group_inputs = ZerosMapper._params_parser(raw_params, args_name, trainable_params)

        # 对于某些从ATen算子中获取的参数，无法直接用于MindSpore算子中，需要进行相应的处理。
        dtype = args.get("dtype")
        args["dtype"] = PYTORCH_MS_MAP["default"] if dtype is None else PYTORCH_MS_MAP[dtype]

        # 该算子在`__init__`中的代码。
        init_template_list = [f"self.{{{variable_slot}}}_{arg_name} = {{{arg_name}}}" for arg_name in args]

        # 获取`parameters_declared`，多数情况下为固定代码，不需要修改。
        parameters_declared = ZerosMapper.generate_parameters_declared(variable_slot, init_template_list, args,
                                                                       trainable_params)
        # 根据`raw_args_name_list和`inputs`的长度关系来确定生成`shape`还是`(shape0, shape1, shape2,...)`。
        diff = len(inputs) - len(raw_args_name_list)
        if diff > 0:
            construct_template = f"opt_{{{variable_slot}}} = ms_np.{op}" \
                                 f"(({', '.join(inputs[:diff + 1])}), self.{{{variable_slot}}}_dtype)"
        else:
            construct_template = f"opt_{{{variable_slot}}} = ms_np.{op}" \
                                 f"({inputs[0]}, self.{{{variable_slot}}}_dtype)"
        # 生成`template`和`exchange_msg`。多数情况下为固定代码，不需要修改。
        template, exchange_msg = reset_template_and_exchange_msg(template, exchange_msg, variable_slot,
                                                                 init_template_list, [construct_template], args,
                                                                 trainable_params, parameters_declared, group_inputs)
        return template, exchange_msg, outputs_list, outputs_mapping

### 新增映射关系文件到工具中，使该映射关系在工具运行中生效。

- 将该映射关系脚本命名为`zeros_mapper`，该文件命名以ATen算子为准。
- 将该映射关系文件放入`mindconverter/graph_based_converter/mapper/aten/prim`目录下，`aten`当中有`nn`，`ops`，`prim`目录，分别对应MindSpore算子的`nn`，`ops`，`numpy`层算子。
- 在[映射关系注册表](../mindconverter/graph_based_converter/mapper/aten_to_ms.json)中增加`"aten::zeros": "mindconverter.graph_based_converter.mapper.aten.prim.zeros_mapper.ZerosMapper"`，

用于动态引用该映射关系文件进行算子转换。

## 验证算子映射关系的正确性

In [2]:
from mindconverter.graph_based_converter.mapper.base import AtenToMindSporeMapper
from mindconverter.graph_based_converter.common.code_fragment import Fragment


def test_mapper(aten_info, inputs_code):
    """
    Test mapper.

    Args:
        aten_info (dict): ATen operator info.
        {
            'op_name': op_name,
            'attribute': dict(),
            'weights': [NodeWeight(),...]
        }
    """
    template, exchange_msg, outputs_list, outputs_mapping = AtenToMindSporeMapper.convert(aten_info["op_name"],
                                                                                          aten_info["attribute"],
                                                                                          aten_info["weights"])
    exchange_msg['var_0']['variable_name'] = 'ms_zeros'
    exchange_msg['var_0']['inputs'] = inputs_code

    fragment = Fragment(data_entity=exchange_msg, code_template=template, outputs=outputs_list,
                        outputs_mapping=outputs_mapping)

    code = fragment()
    init_code = code[0]
    construct_code = code[1]
    print('-' * 30, 'init_code', '-' * 30)
    print('\n'.join(init_code))
    print('-' * 30, 'construct_code', '-' * 30)
    print('\n'.join(construct_code))

In [3]:
aten_info = {
    "op_name": "aten::zeros",
    "attribute": {
        "constant_0": "from_input",
        "constant_1": 4,
        "constant_2": None,
        "constant_3": None,
        "constant_4": None,
        "input_shape": -1,
        "output_shape": (1, 3, 224, 224)},
    "weights": list()
}

test_mapper(aten_info, inputs_code=["x"])

------------------------------ init_code ------------------------------
self.ms_zeros_dtype = mindspore.int64
------------------------------ construct_code ------------------------------
opt_ms_zeros = ms_np.zeros(x, self.ms_zeros_dtype)


In [4]:
aten_info = {
    "op_name": "aten::zeros",
    "attribute": {
        "constant_0[0(*,_,_,_)]": "from_input",
        "constant_1[0(_,*,_,_)]": 3,
        "constant_2[0(_,_,*,_)]": "from_input",
        "constant_3[0(_,_,_,*)]": 224,
        "constant_4": None,
        "constant_5": 4,
        "constant_6": None,
        "constant_7": None,
        "constant_8": None,
        "input_shape": -1,
        "output_shape": (1, 3, 224, 224)
    },
    "weights": list()
}

test_mapper(aten_info, inputs_code=["x", "y"])

------------------------------ init_code ------------------------------
self.ms_zeros_shape1 = 3
self.ms_zeros_shape3 = 224
self.ms_zeros_dtype = mindspore.int64
------------------------------ construct_code ------------------------------
opt_ms_zeros = ms_np.zeros((x, self.ms_zeros_shape1, y, self.ms_zeros_shape3), self.ms_zeros_dtype)
